### Ingestion Schema Manipulation

In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))

if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [2]:
!pip install pyspark==2.4.5

In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [5]:
from pyspark.sql.functions import lit,col,concat
import os

In [33]:
current_dir = os.path.dirname("")
relative_path = "../03 The majestic role of the dataframe/data/Restaurants_in_Wake_County_NC.csv"
absolute_file_path = os.path.join(current_dir, relative_path)

In [34]:
absolute_file_path

'../03 The majestic role of the dataframe/data/Restaurants_in_Wake_County_NC.csv'

In [35]:
df = spark.read.csv(header=True, inferSchema=True, path=absolute_file_path)

In [36]:
print("*** Right after ingestion")
df.show(5)
df.printSchema()

*** Right after ingestion
+--------+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+--------+------------+-----------+-------------+
|OBJECTID|    HSISID|                NAME|            ADDRESS1|ADDRESS2|       CITY|STATE|POSTALCODE|   PHONENUMBER| RESTAURANTOPENDATE|     FACILITYTYPE|PERMITID|           X|          Y|GEOCODESTATUS|
+--------+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+--------+------------+-----------+-------------+
|    1001|4092016024|                WABA|2502 1/2 HILLSBOR...|    null|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18 00:00:00|       Restaurant|    6952|-78.66818477|35.78783803|            M|
|    1002|4092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    null|       CARY|   NC|     27518|(919) 852-6651|2011-11-08 00:00:00|       Food Stand|    6953

In [37]:
print("We have {} records.".format(df.count()))

We have 3440 records.


In [38]:
# Let's transform our dataframe
df =  df.withColumn("county", lit("Wake")) \
        .withColumnRenamed("HSISID", "datasetId") \
        .withColumnRenamed("NAME", "name") \
        .withColumnRenamed("ADDRESS1", "address1") \
        .withColumnRenamed("ADDRESS2", "address2") \
        .withColumnRenamed("CITY", "city") \
        .withColumnRenamed("STATE", "state") \
        .withColumnRenamed("POSTALCODE", "zip") \
        .withColumnRenamed("PHONENUMBER", "tel") \
        .withColumnRenamed("RESTAURANTOPENDATE", "dateStart") \
        .withColumnRenamed("FACILITYTYPE", "type") \
        .withColumnRenamed("X", "geoX") \
        .withColumnRenamed("Y", "geoY") \
        .drop("OBJECTID", "PERMITID", "GEOCODESTATUS")

In [39]:
df = df.withColumn("id",
        concat(col("state"), lit("_"), col("county"), lit("_"), col("datasetId")))

In [41]:
print("*** Dataframe transformed")
df.show(5)

*** Dataframe transformed
+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+------------+-----------+------+------------------+
| datasetId|                name|            address1|address2|       city|state|       zip|           tel|          dateStart|             type|        geoX|       geoY|county|                id|
+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+------------+-----------+------+------------------+
|4092016024|                WABA|2502 1/2 HILLSBOR...|    null|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18 00:00:00|       Restaurant|-78.66818477|35.78783803|  Wake|NC_Wake_4092016024|
|4092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    null|       CARY|   NC|     27518|(919) 852-6651|2011-11-08 00:00:00|       Food Stand|-78.78211173|35.73717591|  Wake|NC_Wake_4

In [42]:
dfUsedForBook = df.drop("address2","zip","tel","dateStart","geoX","geoY","address1","datasetId")

In [43]:
dfUsedForBook.show(5, 15)

+---------------+-----------+-----+---------------+------+---------------+
|           name|       city|state|           type|county|             id|
+---------------+-----------+-----+---------------+------+---------------+
|           WABA|    RALEIGH|   NC|     Restaurant|  Wake|NC_Wake_4092...|
|WALMART DELI...|       CARY|   NC|     Food Stand|  Wake|NC_Wake_4092...|
|CAROLINA SUS...|    RALEIGH|   NC|     Restaurant|  Wake|NC_Wake_4092...|
|THE CORNER V...|    RALEIGH|   NC|Mobile Food ...|  Wake|NC_Wake_4092...|
|   SUBWAY #3726|WAKE FOREST|   NC|     Restaurant|  Wake|NC_Wake_4092...|
+---------------+-----------+-----+---------------+------+---------------+
only showing top 5 rows



In [44]:
df.printSchema()

root
 |-- datasetId: long (nullable = true)
 |-- name: string (nullable = true)
 |-- address1: string (nullable = true)
 |-- address2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- dateStart: timestamp (nullable = true)
 |-- type: string (nullable = true)
 |-- geoX: double (nullable = true)
 |-- geoY: double (nullable = true)
 |-- county: string (nullable = false)
 |-- id: string (nullable = true)



In [45]:
dfUsedForBook.show(5)

+--------------------+-----------+-----+-----------------+------+------------------+
|                name|       city|state|             type|county|                id|
+--------------------+-----------+-----+-----------------+------+------------------+
|                WABA|    RALEIGH|   NC|       Restaurant|  Wake|NC_Wake_4092016024|
|  WALMART DELI #2247|       CARY|   NC|       Food Stand|  Wake|NC_Wake_4092021693|
|CAROLINA SUSHI &a...|    RALEIGH|   NC|       Restaurant|  Wake|NC_Wake_4092017012|
|THE CORNER VENEZU...|    RALEIGH|   NC|Mobile Food Units|  Wake|NC_Wake_4092030288|
|        SUBWAY #3726|WAKE FOREST|   NC|       Restaurant|  Wake|NC_Wake_4092015530|
+--------------------+-----------+-----+-----------------+------+------------------+
only showing top 5 rows



In [46]:
dfUsedForBook.printSchema()

root
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)
 |-- county: string (nullable = false)
 |-- id: string (nullable = true)



In [48]:
print("*** Looking at partitions")
partitionCount = df.rdd.getNumPartitions()
print("Partition count before repartition: {}".format(partitionCount))

*** Looking at partitions
Partition count before repartition: 1


In [53]:
print("*** Update to use more partitions")
df = df.repartition(1000000)
print("Partition count after repartition: {}".format(df.rdd.getNumPartitions()))

*** Update to use more partitions
Partition count after repartition: 1000000
